In [4]:
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")


StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 6, Finished, Available, Finished)

In [5]:
df_crm = spark.read.table("crm_data")
df_erp = spark.read.table("erp_data")

StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 7, Finished, Available, Finished)

In [6]:
display(df_crm)

StatementMeta(, 16d6ebad-c425-4de7-a087-6ecb8d5740c3, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, daf53377-8c17-4e8c-b291-d31c899eed50)

In [6]:
display(df_erp)

StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 369bb5e5-4389-46f6-ab36-0c0c32ef2a15)

In [7]:
#total purchase by a customer

query = """select customer_id,customer_name,sum(amount) as total_order_by_customer
from crm_data
where customer_id is not null and customer_name is not null
group by customer_id,customer_name"""

result = spark.sql(query)
result.show()

StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 9, Finished, Available, Finished)

+-----------+-----------------+-----------------------+
|customer_id|    customer_name|total_order_by_customer|
+-----------+-----------------+-----------------------+
|        239|     Maria Fisher|                 366.41|
|        428|      Levi Carter|                  203.6|
|        504|      Kelly Walsh|                 285.68|
|        612|     Robert Hardy|                 400.33|
|        640|      Shaun Hayes|                 244.22|
|        750|     Eric Sweeney|                 493.45|
|       1274| Danielle Hopkins|                 183.84|
|        158|   Laurie Jackson|                 123.83|
|        717|   Kristi Salazar|                  192.3|
|         16|  Jordan Williams|                 236.53|
|        321|   Carolyn Martin|                 319.84|
|        426|     Mary Bradley|                 342.72|
|        866|   William Miller|                 234.49|
|        896|  Christina Allen|                 459.84|
|        977|   Kimberly Lopez|                 

In [8]:
#removing unneccessary columns and filtering data to remove nulls and writing the data to a gold level table

df_gold_crm = df_crm.drop('contact_email','phone_number')

#add columns year and month of purchase




StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 10, Finished, Available, Finished)

In [9]:
display(df_gold_crm)

StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c9b10153-edb7-452a-84cb-cf69b7fc6932)

In [10]:
import pyspark.sql.functions as f
df_gold_crm = df_gold_crm.withColumn('years',f.year(f.to_timestamp('purchase_date', 'yyyy/mm/dd')))

df_gold_crm = df_gold_crm.withColumn('month',f.month(f.to_timestamp('purchase_date', 'yyyy/mm/dd')))

#df_gold_crm.write.format("delta").save("crm_gold")

StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 12, Finished, Available, Finished)

In [13]:
display(df_gold_crm)

StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c1208559-ceb7-41e2-b115-4735be8b5ae2)

In [14]:
df_gold_crm =df_gold_crm.drop('purchase_date')

#df_gold_crm.write.format("delta").save("crm_gold")

StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 16, Finished, Available, Finished)

In [15]:
display(df_gold_crm)

StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e8d3ba57-fcbd-4d80-87df-968d9958f540)

In [17]:
#write to a delta table
df_gold_crm.write.format('delta').save('crm_gold')

StatementMeta(, 337cef2a-7eff-4cc1-90fd-a464a0893bd1, 19, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o12191.save.
: java.util.concurrent.ExecutionException: Operation failed: "Bad Request", 400, HEAD, http://onelake.dfs.fabric.microsoft.com/07cb77f8-4c02-4510-ad7a-373c3e1e089c/6748858d-7cf9-477a-8767-75204539daa8/crm_gold/_delta_log?upn=false&action=getStatus&timeout=90
	at com.google.common.util.concurrent.AbstractFuture$Sync.getValue(AbstractFuture.java:306)
	at com.google.common.util.concurrent.AbstractFuture$Sync.get(AbstractFuture.java:293)
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:116)
	at com.google.common.util.concurrent.Uninterruptibles.getUninterruptibly(Uninterruptibles.java:135)
	at com.google.common.cache.LocalCache$Segment.getAndRecordStats(LocalCache.java:2410)
	at com.google.common.cache.LocalCache$Segment.loadSync(LocalCache.java:2380)
	at com.google.common.cache.LocalCache$Segment.lockedGetOrLoad(LocalCache.java:2342)
	at com.google.common.cache.LocalCache$Segment.get(LocalCache.java:22